In [8]:
import os
import cv2

# Rutas
img_dir = "dataset/UR-Fall/valid/images"
label_dir = "dataset/UR-Fall/valid/labels"
output_dir = "dataset/UR-Fall/valid/view"
os.makedirs(output_dir, exist_ok=True)

# Colores para cada clase (ajustable si hay más clases)
colors = {
    0: (0, 255, 0),    # Verde para no-fall
    1: (0, 0, 255),    # Rojo para fall
}

# Obtener primeras 200 imágenes
image_files = sorted([
    f for f in os.listdir(img_dir)
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
])[:200]

for img_file in image_files:
    img_path = os.path.join(img_dir, img_file)
    label_file = os.path.splitext(img_file)[0] + ".txt"
    label_path = os.path.join(label_dir, label_file)

    if not os.path.exists(label_path):
        print(f"Label no encontrado para {img_file}, saltando.")
        continue

    # Cargar imagen
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    # Leer y dibujar cada bounding box
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            class_id, cx, cy, bw, bh = map(float, parts)
            color = colors.get(int(class_id), (255, 255, 0))

            # Convertir coordenadas YOLO a píxeles
            x1 = int((cx - bw / 2) * w)
            y1 = int((cy - bh / 2) * h)
            x2 = int((cx + bw / 2) * w)
            y2 = int((cy + bh / 2) * h)

            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, f'Class {int(class_id)}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Guardar imagen con cajas
    output_path = os.path.join(output_dir, img_file)
    cv2.imwrite(output_path, img)

print("✅ 200 imágenes guardadas con bounding boxes.")

✅ 200 imágenes guardadas con bounding boxes.


In [3]:
import os

# Ruta base al dataset
base_path = "dataset/Merged_Dataset"
splits = ["train", "valid", "test"]

fall_count = 0
no_fall_count = 0
archivos_sin_etiquetas = 0

for split in splits:
    label_dir = os.path.join(base_path, split, "labels")

    for file in os.listdir(label_dir):
        if file.endswith(".txt"):
            path = os.path.join(label_dir, file)
            with open(path, 'r') as f:
                lines = f.readlines()
                if not lines:
                    archivos_sin_etiquetas += 1
                    print(path)
                for line in lines:
                    class_id = int(line.strip().split()[0])
                    if class_id == 1:
                        fall_count += 1
                    elif class_id == 0:
                        no_fall_count += 1
                    else:
                        print(class_id)
                        print(path)
        else:
            print(file)

print("📊 RESULTADOS TOTALES:")
print(f"🔥 Fall (class_id = 1): {fall_count}")
print(f"📦 No Fall (class_id = 0): {no_fall_count}")
print(f"⚠️ Archivos sin etiquetas: {archivos_sin_etiquetas}")

📊 RESULTADOS TOTALES:
🔥 Fall (class_id = 1): 11808
📦 No Fall (class_id = 0): 20638
⚠️ Archivos sin etiquetas: 0


In [6]:
import os

base_path = "dataset/UR-Fall"
splits = ["train", "valid", "test"]
image_exts = [".jpg", ".png", ".jpeg"]

def invertir_label(line):
    parts = line.strip().split()
    if parts:
        class_id = parts[0]
        if class_id == "0":
            parts[0] = "1"
        elif class_id == "1":
            parts[0] = "0"
    return " ".join(parts)

for split in splits:
    labels_dir = os.path.join(base_path, split, "labels")
    images_dir = os.path.join(base_path, split, "images")

    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        label_path = os.path.join(labels_dir, label_file)

        with open(label_path, "r") as f:
            lines = f.readlines()

        # Si el label está vacío, eliminar .txt y su imagen correspondiente
        if not lines:
            os.remove(label_path)
            image_name = label_file.rsplit(".", 1)[0]
            for ext in image_exts:
                img_path = os.path.join(images_dir, image_name + ext)
                if os.path.exists(img_path):
                    os.remove(img_path)
                    break
            print(f"🗑️ Eliminado label vacío y su imagen: {label_file}")
            continue

        # Invertir los labels
        new_lines = [invertir_label(line) + "\n" for line in lines]

        # Sobrescribir con los labels invertidos
        with open(label_path, "w") as f:
            f.writelines(new_lines)

        print(f"🔁 Labels invertidos en: {label_file}")

🗑️ Eliminado label vacío y su imagen: adl-01-cam0-rgb-003_png.rf.a38972b356f98322b92df9d1daf6d2c6.txt
🗑️ Eliminado label vacío y su imagen: adl-01-cam0-rgb-004_png.rf.c30f89c199ef9b50515ab1fd37ba885a.txt
🔁 Labels invertidos en: adl-01-cam0-rgb-006_png.rf.9577d9a88db9fbff778c0d8486fec183.txt
🔁 Labels invertidos en: adl-01-cam0-rgb-007_png.rf.2bc4fb606f6de8a9b577c0f678469173.txt
🔁 Labels invertidos en: adl-01-cam0-rgb-008_png.rf.87f92860a5d59746c21f8247f4d9a12f.txt
🔁 Labels invertidos en: adl-01-cam0-rgb-010_png.rf.09f74f5ea67d80687accf5d15aae6a34.txt
🔁 Labels invertidos en: adl-01-cam0-rgb-012_png.rf.e69d64866fb5c17eb38bb14f506fbb0f.txt
🔁 Labels invertidos en: adl-01-cam0-rgb-015_png.rf.1d0797945e6439baac5c2f050c1d8cd1.txt
🔁 Labels invertidos en: adl-01-cam0-rgb-017_png.rf.8c12a68fd14cdd1cac677d1e6500854a.txt
🔁 Labels invertidos en: adl-01-cam0-rgb-018_png.rf.e9677d5f34e22e645d58ac2f82adeb0e.txt
🔁 Labels invertidos en: adl-01-cam0-rgb-019_png.rf.a324106703ae30a3ba7ee6b48da89c06.txt
🔁 La

In [2]:
import os

base_path = "dataset/Merged_Dataset"  # Cambia si tu carpeta raíz tiene otro nombre
splits = ["train", "valid", "test"]
image_exts = [".jpg", ".jpeg", ".png"]
errores = {
    "sin_label": [],
    "label_vacio": [],
    "class_id_invalido": []
}

def verificar_labels(split):
    labels_path = os.path.join(base_path, split, "labels")
    images_path = os.path.join(base_path, split, "images")

    image_files = [f for f in os.listdir(images_path) if f.lower().endswith(tuple(image_exts))]

    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = base_name + ".txt"
        label_path = os.path.join(labels_path, label_file)

        # Verificar existencia del label
        if not os.path.exists(label_path):
            errores["sin_label"].append(os.path.join(split, "images", img_file))
            continue

        # Verificar contenido del label
        with open(label_path, "r") as f:
            lines = [line.strip() for line in f if line.strip()]

        if not lines:
            errores["label_vacio"].append(os.path.join(split, "labels", label_file))
            continue

        for line in lines:
            parts = line.split()
            if not parts:
                continue
            class_id = parts[0]
            if class_id not in ["0", "1"]:
                errores["class_id_invalido"].append(os.path.join(split, "labels", label_file))
                break

# Ejecutar verificación en cada split
for s in splits:
    verificar_labels(s)

# Mostrar resultados
print("\n📝 RESULTADOS:")
for error_type, archivos in errores.items():
    if archivos:
        print(f"\n❌ {error_type.replace('_', ' ').capitalize()} ({len(archivos)} archivos):")
        for f in archivos:
            print(f" - {f}")
    else:
        print(f"\n✅ No se encontraron errores de tipo: {error_type.replace('_', ' ').capitalize()}")


📝 RESULTADOS:

✅ No se encontraron errores de tipo: Sin label

✅ No se encontraron errores de tipo: Label vacio

✅ No se encontraron errores de tipo: Class id invalido


In [ ]:
import os

# Ruta base al dataset
base_path = "dataset/Merged_Dataset"
splits = ["train", "valid", "test"]

# Contadores globales
total_fall = 0
total_no_fall = 0
total_sin_etiquetas = 0

for split in splits:
    fall_count = 0
    no_fall_count = 0
    sin_etiquetas = 0

    label_dir = os.path.join(base_path, split, "labels")
    print(f"\n📂 Analizando carpeta: {split}")

    for file in os.listdir(label_dir):
        if file.endswith(".txt"):
            path = os.path.join(label_dir, file)
            with open(path, 'r') as f:
                lines = [line.strip() for line in f if line.strip()]
                if not lines:
                    sin_etiquetas += 1
                    print(f"⚠️ Vacío: {path}")
                for line in lines:
                    try:
                        class_id = int(line.split()[0])
                        if class_id == 1:
                            fall_count += 1
                        elif class_id == 0:
                            no_fall_count += 1
                        else:
                            print(f"❗ class_id inválido ({class_id}) en {path}")
                    except ValueError:
                        print(f"❌ Error de lectura en línea: '{line}' en {path}")
        else:
            print(f"🧾 Archivo ignorado: {file}")

    total_fall += fall_count
    total_no_fall += no_fall_count
    total_sin_etiquetas += sin_etiquetas

    print(f"📊 {split.upper()} - Fall (1): {fall_count}, No Fall (0): {no_fall_count}, Vacíos: {sin_etiquetas}")

# Totales
print("\n✅ RESUMEN GLOBAL:")
print(f"🔥 Fall (class_id = 1): {total_fall}")
print(f"📦 No Fall (class_id = 0): {total_no_fall}")
print(f"⚠️ Archivos sin etiquetas: {total_sin_etiquetas}")


📂 Analizando carpeta: train



KeyboardInterrupt


KeyboardInterrupt

